In [1]:
import dask as da
from dask.distributed import LocalCluster, Client
import glob
import numpy as np
import pandas as pd
%pylab inline
import scipy.ndimage as ndimage
import xarray as xr

Populating the interactive namespace from numpy and matplotlib


In [2]:
cluster = LocalCluster(processes=False, local_dir= "/g/data1a/e14/cp3790/dask-workers")
client = Client(cluster)
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/worker.py:475: UserWarning: The local_dir keyword has moved to local_directory
  warnings.warn("The local_dir keyword has moved to local_directory")
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/contextlib.py:88: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Client Scheduler: inproc://10.0.64.28/18062/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 8 Memory: 33.67 GB


In [3]:
# Opens Climatology and threshold files, that have been created before.

localDir = "/g/data/e14/cp3790/Charuni/"
thresh = xr.open_dataarray(localDir + 'threshold-region.nc')
clim = xr.open_dataarray(localDir + 'clim-single-region.nc')

In [19]:
# Opens ERA5 daily tmax data

files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/ERA5-new/era5_dailytmax_*.nc'))


era5_dailytmax_aus = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('2010', '2018'), longitude=slice(140, 145), latitude=slice(-32, -34))
obs_tmax = era5_dailytmax_aus["dmax"].load() #.sel(time=~((era5_dailytmax_aus["dmax"].time.dt.month == 2) & (era5_dailytmax_aus["dmax"].time.dt.day == 29)))
obs_tmax.attrs['units'] = 'deg C'

In [20]:
obs_tmax

<xarray.DataArray 'dmax' (time: 3287, latitude: 9, longitude: 21)>
array([[[36.99035 , 37.22445 , 36.820824, ..., 31.041101, 30.432459,
         29.723719],
        [36.578667, 36.72558 , 37.3536  , ..., 31.265528, 30.639124,
         29.770533],
        [37.048485, 36.95159 , 37.382652, ..., 31.688501, 30.748896,
         29.643   ],
        ...,
        [37.83956 , 38.21734 , 38.21734 , ..., 33.18508 , 32.31167 ,
         31.564173],
        [37.44561 , 37.752373, 37.891197, ..., 33.64361 , 32.726585,
         31.956507],
        [36.07012 , 36.376854, 36.95968 , ..., 34.613884, 33.52575 ,
         32.400475]],

       [[27.017908, 26.37536 , 26.191309, ..., 30.54223 , 29.818964,
         29.271662],
        [25.510004, 24.80932 , 24.851313, ..., 30.643946, 29.943293,
         29.44604 ],
        [24.61883 , 24.381495, 24.530024, ..., 30.319452, 30.07406 ,
         29.601007],
        ...,
        [25.865198, 25.600428, 25.422815, ..., 28.462824, 28.637201,
         28.7615  ],
     

In [21]:
hw_days = obs_tmax.groupby('time.dayofyear') > thresh 

In [22]:
hw_days

<xarray.DataArray (time: 3287, latitude: 9, longitude: 21)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, 

In [23]:
# Creating new coordinate for event duration 

hw_days = hw_days.assign_coords({'event_duration': hw_days.dayofyear*0})